<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

In [64]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import datetime


In [85]:
data = pd.read_csv('data.csv')
X = data[['f1', 'f2']]
y = data.label
print(X.shape, y.shape)
data.head()

(20000, 2) (20000,)


,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [66]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

validation_data = x_test
validation_target = y_test

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(14000, 2) (14000,)
(6000, 2) (6000,)


In [101]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(2,)),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomUniform(0,1))
  ])

In [102]:
# class MyCallback(tf.keras.callbacks.Callback):
    
#     def on_train_begin(self, logs={}):
#         ## on begin of training, we are creating an instance varible called history
#         ## it is a dict with keys [loss, acc, val_loss, val_acc]
#         self.history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}
        
#     def on_epoch_end(self, epoch, logs={}):
#         ## on end of each epoch, we will get logs and update the self.history dict
#         self.history['loss'].append(logs.get('loss'))
#         self.history['acc'].append(logs.get('acc'))
#         if logs.get('val_loss', -1) != -1:
#             self.history['val_loss'].append(logs.get('val_loss'))
#         if logs.get('val_acc', -1) != -1:
#             self.history['val_acc'].append(logs.get('val_acc'))
            
# history_own = MyCallback()

In [117]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        
        _val_f1 = f1_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        
        print("f1 score:", _val_f1)
        returnvalidation_data
    
    
f1_sco = Metrics()

# model.fit(training_data, training_target,
#           validation_data=(validation_data, validation_target),
#           nb_epoch=10,
#           batch_size=64,
#           callbacks=[metrics])

In [119]:
model = create_model()

model.compile(optimizer = tf.keras.optimizers.SGD(momentum=0.9),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

# log_dir="logs\\sgd_moment\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="logs\\sgd_moment\\"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads=True)

# model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])
model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test), callbacks=[f1_sco])

Train on 14000 samples, validate on 6000 samples
Epoch 1/5
13792/14000 [============================>.] - ETA: 0s - loss: 0.7277 - acc: 0.5125

AttributeError: 'Sequential' object has no attribute 'validation_data'

In [ ]:
# # !tasklist
# !taskkill /F /PID 1676

In [12]:
# %load_ext tensorboard
# %tensorboard --logdir logs/sgd_moment
# # %tensorboard --logdir logs/sgd_moment --host localhost

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>